# Goedaardig vs kwaadaardige tumoren - Support Vector Machines

We voorspellen of een tumor goedaardig (_benign_ of kortweg _B_) is, of de patiënt weldegelijk borstkanker heeft en de tumor dus kwaardaardig (_malignant_ of kortweg _M_) is.

In [19]:
import numpy as np
# data inladen
import pandas as pd
from pandas import read_csv
# data opsplitsen in trainings-, validatie- en testset
from sklearn.model_selection import train_test_split
# grafieken
from matplotlib import pyplot as plt
# nodig voor het maken van het svm model
from sklearn import svm
# nodig om accuraatheid van de voorspellingen te bekijken
from sklearn import metrics

In [20]:
# We laden de data
gegevens = read_csv("data.csv")

# We genereren een random random_state
random_state = np.random.randint(100)

# We verdelen de data in 50% training, 25% validatie en 25% test
tmp, testData = train_test_split(gegevens, test_size=0.25, random_state=random_state)
trainingData, validatieData = train_test_split(tmp, test_size=0.33, random_state=random_state)

# Print the shapes of the resulting datasets
print(f"Grootte training data: \t{trainingData.shape} (50%)")
print(f"Grootte validatie data: {validatieData.shape} (25%)")
print(f"Grootte test data: \t{testData.shape} (25%)")


Grootte training data: 	(285, 33) (50%)
Grootte validatie data: (141, 33) (25%)
Grootte test data: 	(143, 33) (25%)


In [21]:
print(trainingData.head())

           id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
394    903554         B        12.10         17.72           78.07      446.2   
372   9012795         M        21.37         15.10          141.30     1386.0   
14   84667401         M        13.73         22.61           93.60      578.3   
333    897137         B        11.25         14.78           71.38      390.0   
397  90401602         B        12.80         17.46           83.05      508.3   

     smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
394          0.10290           0.09758        0.047830             0.033260   
372          0.10010           0.15150        0.193200             0.125500   
14           0.11310           0.22930        0.212800             0.080250   
333          0.08306           0.04458        0.000974             0.002941   
397          0.08044           0.08895        0.073900             0.040830   

     ...  texture_worst  perimeter_wor

In [22]:
# We tellen het aantal diagnoses in de training data
diagnosesTotaal = trainingData['diagnosis'].value_counts()

print(diagnosesTotaal)

aantalGoedaardig = diagnosesTotaal[0]
aantalKwaadaardig = diagnosesTotaal[1]

print(f"Er zijn {aantalGoedaardig} goedaardige tumoren en {aantalKwaadaardig} kwaadaardige tumoren in de training data.")

B    181
M    104
Name: diagnosis, dtype: int64
Er zijn 181 goedaardige tumoren en 104 kwaadaardige tumoren in de training data.


In [23]:
# We selecteren de kolommen met de features (x-waarden)
trainingFeatures = trainingData.iloc[:,2:32]
validatieFeatures = validatieData.iloc[:,2:32]
testFeatures = testData.iloc[:,2:32]

# We selecteren de kolom met de diagnoses (y-waarden)
trainingDiagnoses = trainingData.iloc[:,1]
validatieDiagnoses = validatieData.iloc[:,1]
testDiagnoses = testData.iloc[:,1]

In [24]:
print(trainingFeatures)

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
394       12.100         17.72           78.07      446.2          0.10290   
372       21.370         15.10          141.30     1386.0          0.10010   
14        13.730         22.61           93.60      578.3          0.11310   
333       11.250         14.78           71.38      390.0          0.08306   
397       12.800         17.46           83.05      508.3          0.08044   
..           ...           ...             ...        ...              ...   
473       12.270         29.97           77.42      465.4          0.07699   
259       15.530         33.56          103.70      744.9          0.10630   
522       11.260         19.83           71.30      388.1          0.08511   
244       19.400         23.50          129.10     1155.0          0.10270   
505        9.676         13.14           64.12      272.5          0.12550   

     compactness_mean  concavity_mean  concave points_mean  sym

In [25]:
print(trainingDiagnoses)

394    B
372    M
14     M
333    B
397    B
      ..
473    B
259    M
522    B
244    M
505    B
Name: diagnosis, Length: 285, dtype: object


In [26]:
# we halen de waardes van de feautures op
trainingX = trainingFeatures.values
validatieX = validatieFeatures.values
testX = testFeatures.values

print(trainingX)

[[1.210e+01 1.772e+01 7.807e+01 ... 6.266e-02 3.049e-01 7.081e-02]
 [2.137e+01 1.510e+01 1.413e+02 ... 1.966e-01 2.730e-01 8.666e-02]
 [1.373e+01 2.261e+01 9.360e+01 ... 2.208e-01 3.596e-01 1.431e-01]
 ...
 [1.126e+01 1.983e+01 7.130e+01 ... 2.832e-02 2.557e-01 7.613e-02]
 [1.940e+01 2.350e+01 1.291e+02 ... 1.564e-01 2.920e-01 7.614e-02]
 [9.676e+00 1.314e+01 6.412e+01 ... 1.075e-01 2.848e-01 1.364e-01]]


In [27]:
# we halen de waardes van de diagnoses op
trainingY = trainingDiagnoses.values
validatieY = validatieDiagnoses.values
testY = testDiagnoses.values

print(trainingY)

['B' 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B'
 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'M' 'M' 'M' 'B' 'B'
 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B'
 'B' 'M' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B'
 'M' 'B' 'M' 'M' 'B' 'M' 'M' 'M' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'B'
 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M'
 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'M' 'M' 'M'
 'B' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B'
 'M' 'B' 'M' 'M' 'M' 'M' 'B' 'B' 'M' 'M' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'M'
 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M'
 'M' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M'
 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'M' 'M' 'B' 'M'
 'M' 'B' 'B' 'M' 'M' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'B

In [28]:
# Dit is de classifier of classificator
# We gebruiken de kernel trick om de data te kunnen scheiden
# In dit geval gebruiken we een lineaire kernel
clf = svm.SVC(kernel='linear')

# We trainen de classifier met de training data
clf.fit(trainingX,trainingY)

# We voorspellen de diagnoses van de validatie data
validatieVoorspellingen = clf.predict(validatieX)

print(validatieVoorspellingen)

['B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B'
 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'B'
 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'B'
 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B'
 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B'
 'B' 'M' 'B' 'B' 'M' 'M' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B'
 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'M' 'B'
 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B']


In [29]:
# we berekenen de accuraatheid van de voorspellingen
accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)
print(f"De accuraatheid van de voorspellingen is {accuraatheid*100:.2f}%")

De accuraatheid van de voorspellingen is 92.20%


### Validatiegegevens gebruiken om de 3 meta-parameters te bepalen.

#### C-term<br>
De C-term staat voor de __foutentoleratie__ van het model.<br><br>
*Kleine waarden voor C:* <br>
Wanneer C klein is, laat het model meer fouten toe, waardoor de beslissingsgrens eenvoudiger wordt.<br><br>
*Grote waarden voor C:* <br>
Wanneer C groot is, laat het model een laag aantal fouten toe, waardoor de beslissingsgrens complexer wordt.<br>

In [30]:
# De C-term is een metaparameter en staat voor de foutentoleratie van het model.
mogelijkeCs = [c for c in range(1, 10)]

hoogsteAccuraatheid = 0
cOpt = 0

# Overloop verschillende waarden voor C
for C in mogelijkeCs:

    clf = svm.SVC(kernel='linear', C=C)
    clf.fit(trainingX, trainingY)
    validatieVoorspellingen = clf.predict(validatieX)
    accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)

    print(f"De accuraatheid van de voorspellingen op de validatiedata is {accuraatheid*100:.5f}% met C={C}")

    if accuraatheid > hoogsteAccuraatheid:
        hoogsteAccuraatheid = accuraatheid
        cOpt = C

print(f"\nDe optimale waarde voor C is {cOpt} met een accuraatheid van {hoogsteAccuraatheid*100:.2f}%")

De accuraatheid van de voorspellingen op de validatiedata is 92.19858% met C=1
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met C=3
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met C=4
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met C=5
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met C=6
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met C=7
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met C=8
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met C=9

De optimale waarde voor C is 2 met een accuraatheid van 95.04%


#### Kernel<br>
De kernel zegt het model welke 'trick' het het moet gebruiken om de data te scheiden.

In [31]:
# De kernel zegt het model welke 'trick' het het moet gebruiken om de data te scheiden.
mogelijkeKernels = ['linear', 'poly', 'rbf', 'sigmoid']

optKernel = ""
hoogsteAccuraatheid = 0

for kernel in mogelijkeKernels:

    clf = svm.SVC(kernel=kernel, C=cOpt)
    clf.fit(trainingX, trainingY)
    validatieVoorspellingen = clf.predict(validatieX)
    accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)

    print(f"De accuraatheid van de voorspellingen op de validatiedata is {accuraatheid*100:.5f}% met de kernel '{kernel}' en C={cOpt}")

    if accuraatheid > hoogsteAccuraatheid:
        hoogsteAccuraatheid = accuraatheid
        optKernel = kernel

print(f"\nDe optimale kernel is '{optKernel}' met een accuraatheid van {hoogsteAccuraatheid*100:.2f}% voor C={cOpt}")

De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 87.23404% met de kernel 'poly' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 90.07092% met de kernel 'rbf' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 42.55319% met de kernel 'sigmoid' en C=2

De optimale kernel is 'linear' met een accuraatheid van 95.04% voor C=2


#### Gamma<br>
De $\gamma$ staat voor de __foutentoleratie__ van het model.<br><br>
*Kleine waarden voor $\gamma$:* <br>
Een te lage gamma-waarde kan leiden tot _overgeneralisatie_, waarbij de beslissingsgrens te eenvoudig is en het model niet goed presteert op de trainingsgegevens.<br><br>
*Grote waarden voor $\gamma$:* <br>
Een te hoge gamma-waarde kan leiden tot _overfitting_, waarbij het model de trainingsgegevens te nauw volgt en dus niet goed zal presteren op nog niet eerder geziene gegevens.<br>

In [32]:
# De C-term is een metaparameter en staat voor de foutentoleratie van het model.
mogelijkeGammas = [gamma for gamma in range(1, 10)]

hoogsteAccuraatheid = 0
gammaOpt = 0

# Overloop verschillende waarden voor C
for gamma in mogelijkeGammas:

    clf = svm.SVC(kernel=optKernel, C=cOpt, gamma=gamma)
    clf.fit(trainingX, trainingY)
    validatieVoorspellingen = clf.predict(validatieX)
    accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)

    print(f"De accuraatheid van de voorspellingen op de validatiedata is {accuraatheid*100:.5f}% met gamma={gamma}, de kernel '{optKernel}' en C={cOpt}")

    if accuraatheid > hoogsteAccuraatheid:
        hoogsteAccuraatheid = accuraatheid
        gammaOpt = gamma

print(f"\nDe optimale waarde voor gamma is {gammaOpt} met een accuraatheid van {hoogsteAccuraatheid*100:.2f}% voor de kernel '{optKernel}' en C={cOpt}")

De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=1, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=2, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=3, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=4, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=5, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=6, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=7, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=8, de kernel 'linear' en C=2
De accuraatheid van de voorspellingen op de validatiedata is 95.03546% met gamma=9, de kernel 'linear' en C=2

De optima

### Toepassen op de testdata

In [33]:
clf = svm.SVC(kernel=optKernel, C=cOpt, gamma=gammaOpt)
clf.fit(trainingX, trainingY)
testVoorspellingen = clf.predict(testX)
accuraatheid = metrics.accuracy_score(testY, testVoorspellingen)

print(f"De accuraatheid van de voorspellingen op de testdata is {accuraatheid*100:.2f}% voor de kernel '{optKernel}', C={cOpt} en gamma={gammaOpt}")

De accuraatheid van de voorspellingen op de testdata is 95.80% voor de kernel 'linear', C=2 en gamma=1
